## Prophet Modeling

> **Project:** Houston 311 Detector  
> **Author:** Mojoolu (Mojo) Roberts  
> **Environment:** `.venv`  
> **Data Contract:** All notebooks read from raw Houston 311 data that can be downloaded here (link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from prophet import Prophet
from sqlalchemy import create_engine
import logging
import os
from dotenv import load_dotenv

load_dotenv()
db_url = os.getenv("DATABASE_URL")
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
pd.set_option("display.float_format", "{:.2f}".format)

In [2]:
# Connect to your Postgres database
engine = create_engine(db_url)

# Load the table into a DataFrame
df = pd.read_sql("SELECT * FROM houston_311", engine)

pio.renderers.default = "notebook_connected"

In [3]:
df.describe

<bound method NDFrame.describe of                    CASE NUMBER                                 NEIGHBORHOOD  \
0        12091834-101002444724                 EAST LITTLE YORK / HOMESTEAD   
1        12091835-101002444725                          NORTHSIDE/NORTHLINE   
2                 101002444726                                     MID WEST   
3                 101002444727  WASHINGTON AVENUE COALITION / MEMORIAL PARK   
4        12091836-101002444730                               GREATER UPTOWN   
...                        ...                                          ...   
3471667             2500463040                        SPRING BRANCH CENTRAL   
3471668             2500463039                         INDEPENDENCE HEIGHTS   
3471669             2500463038                                KINGWOOD AREA   
3471670             2500463037                             ADDICKS PARK TEN   
3471671             2500463036                         INDEPENDENCE HEIGHTS   

                 

In [4]:
df.dtypes

CASE NUMBER                     object
NEIGHBORHOOD                    object
DEPARTMENT                      object
DIVISION                        object
CASE TYPE                       object
CREATED DATE            datetime64[ns]
CLOSED DATE             datetime64[ns]
LATITUDE                       float64
LONGITUDE                      float64
CATEGORY                        object
RESOLUTION_TIME_DAYS           float64
dtype: object

In [5]:
cols = ["NEIGHBORHOOD", "DEPARTMENT", "DIVISION", "CATEGORY", "CASE TYPE"]

for c in cols:
    df[c] = (
        df[c]
        .astype(str)
        .str.strip()
        .str.title()
        .str.replace("\s+", " ", regex=True)
    )

## Forecast Citywide 311 Complaint Volume (Prophet)

**Purpose**  
Model and forecast monthly 311 complaint volume at the citywide level.

**Data Preparation**
- Aggregate complaints to month-end frequency using `CREATED DATE`
- Rename columns to Prophet’s required schema:
  - `ds`: timestamp
  - `y`: complaint count
- Exclude months with insufficient data to avoid unstable signals
- Remove the most recent incomplete month to prevent partial-month bias

**Modeling Approach**
- Prophet time-series model
- 95% uncertainty interval
- Daily seasonality enabled to capture intra-month effects

**Forecast**
- Generate predictions for the next 12 months
- Output includes trend, seasonality, and uncertainty bounds

**Why This Matters**
- Establishes a citywide baseline forecast
- Serves as a reference for neighborhood- and category-level models
- Helps validate whether trends observed in EDA persist forward

**Notes**
- This model operates on volume only (not resolution time)
- Results are exploratory; production forecasts are generated separately


In [6]:
overall_ts = (
    df.groupby(pd.Grouper(key="CREATED DATE", freq="ME"))
      .size()
      .reset_index(name="y")
      .rename(columns={"CREATED DATE": "ds"})
)

cleaned_ts = overall_ts[overall_ts["y"] > 1000].sort_values("ds")

cleaned_ts = cleaned_ts.iloc[:-1] # drop last month

m = Prophet(interval_width=0.95, yearly_seasonality=True)
m.fit(cleaned_ts)

future = m.make_future_dataframe(periods=12, freq="ME")
forecast = m.predict(future)

cutoff = cleaned_ts["ds"].max()
fc = forecast[forecast["ds"] > cutoff]

combined = pd.concat([
    cleaned_ts[["ds", "y"]].rename(columns={"y": "value"}),
    fc[["ds", "yhat"]].rename(columns={"yhat": "value"}),
]).sort_values("ds")

combined["Rolling_3M_Full"] = combined["value"].rolling(3, min_periods=1).mean()

combined_plot = combined[combined["ds"] >= (cutoff - pd.offsets.MonthEnd(2))].copy()
combined_plot.loc[combined_plot["ds"] <= cutoff, "Rolling_3M_Full"] = None

fig = go.Figure()

# Actuals
fig.add_trace(go.Scatter(
    x=cleaned_ts["ds"],
    y=cleaned_ts["y"],
    mode="lines+markers",
    name="Actuals",
    line=dict(width=2)
))

# Forecast line
fig.add_trace(go.Scatter(
    x=fc["ds"],
    y=fc["yhat"],
    mode="lines",
    name="Forecast",
    line=dict(width=3)
))

# Confidence interval band
fig.add_trace(go.Scatter(
    x=pd.concat([fc["ds"], fc["ds"][::-1]]),
    y=pd.concat([fc["yhat_upper"], fc["yhat_lower"][::-1]]),
    fill="toself",
    fillcolor="rgba(255,255,255,0.12)",
    line=dict(color="rgba(255,255,255,0)"),
    hoverinfo="skip",
    name="95% CI"
))

fig.add_trace(go.Scatter(
    x=combined_plot["ds"],
    y=combined_plot["Rolling_3M_Full"],
    mode="lines",
    name="3M Rolling Trend (Actuals → Forecast)",
    line=dict(width=2, dash="dash")
))

fig.update_layout(
    title=f"Monthly 311 Complaint Forecast",
    template="plotly_dark",
    height=700,
    hovermode="x unified",
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    font_color="#FFF",
    hoverlabel=dict(
        bgcolor="rgba(20,20,20,0.95)",
        font_color="#EAEAEA",
        bordercolor="#555",
        font_size=13,
        font_family="Inter, Arial, sans-serif",
    )
)

fig.show()

20:56:52 - cmdstanpy - INFO - Chain [1] start processing
20:56:52 - cmdstanpy - INFO - Chain [1] done processing


## Evaluate Forecast Accuracy (In-Sample)

**Purpose**  
Assess how well the Prophet model fits historical 311 complaint volume.

**Method**
- Merge forecasted values with observed historical data
- Restrict evaluation to the training (historical) period
- Compare actual counts (`y`) against model predictions (`yhat`)

**Metrics Reported**
- **MAE (Mean Absolute Error):** average absolute deviation in complaint counts
- **RMSE (Root Mean Squared Error):** penalizes larger errors more heavily
- **MAPE (Mean Absolute Percentage Error):** relative error expressed as a percentage

**Why This Matters**
- Quantifies model fit before trusting future projections
- Provides a baseline for comparing alternative models or configurations
- Helps determine whether the model is sufficiently reliable for interpretation

**Notes**
- Metrics are in-sample and may be optimistic
- Out-of-sample validation is recommended for production use


In [7]:
# Merge forecasted values with actuals
results = cleaned_ts.merge(forecast, on='ds', how='left')

# Keep only training portion (historical)
historical = results[results['ds'] <= cleaned_ts['ds'].max()]

In [8]:
# Calculate error metrics
y_true = historical['y']
y_pred = historical['yhat']

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

MAE:  2693.89
RMSE: 3733.17
MAPE: 8.28%


In [9]:
# Keep only periods that exist in your original data
actual = cleaned_ts.set_index("ds")
predicted = forecast.set_index("ds")[["yhat", "yhat_lower", "yhat_upper"]]

# Join actuals with predictions
compare = actual.join(predicted, how="left")

# Compute residuals (errors)
compare["error"] = compare["y"] - compare["yhat"]
compare.head()

,y,yhat,yhat_lower,yhat_upper,error
ds,,,,,
2017-01-31,33903,31350.35,24045.48,38382.91,2552.65
2017-02-28,26991,27710.23,20319.18,34903.09,-719.23
2017-03-31,30619,29617.02,22617.44,36564.35,1001.98
2017-04-30,27820,29095.53,21739.49,36484.76,-1275.53
2017-05-31,28097,32411.87,25357.85,39922.64,-4314.87


## Neighborhood-Level Forecast with Confidence Intervals and Extended Rolling Trend

**Purpose**  
Visualize monthly 311 complaint forecasts for the highest-volume neighborhoods, including uncertainty bounds and a smoothed trend that extends seamlessly from historical data into the forecast horizon.

**Data Preparation**
- Aggregate complaints to monthly counts per neighborhood
- Remove low-volume months and incomplete trailing months
- Require a minimum history length to ensure model stability

**Modeling**
- Fit an independent Prophet model per neighborhood
- Monthly frequency with yearly seasonality
- Forecast 12 months ahead

**Visualization Components**
- **Actuals:** historical monthly complaint counts
- **Forecast:** Prophet point estimates for future months
- **Confidence Intervals:** 95% uncertainty band around forecasts
- **Rolling Trend:** 3-month rolling average computed on combined actuals and forecast values
  - Rolling trend bridges the last observed month into the forecast period
  - Only valid rolling values beyond the historical cutoff are displayed

**Why This Matters**
- Provides continuity between historical trends and future projections
- Makes forecast direction easier to interpret than raw point estimates alone
- Communicates uncertainty explicitly rather than hiding it

**Notes**
- Rolling trends are diagnostic only and are not used for model fitting
- Forecast reliability should be interpreted alongside MAPE values


In [10]:
# Identify top 3 neighborhoods by volume
top_3_neigh = (
    df["NEIGHBORHOOD"]
    .value_counts()
    .head(3)
    .index
    .tolist()
)

results = []

for neigh in top_3_neigh:
    sub = df[df["NEIGHBORHOOD"] == neigh].copy()

    ts = (
        sub
        .groupby(pd.Grouper(key="CREATED DATE", freq="ME"))
        .size()
        .reset_index(name="y")
        .rename(columns={"CREATED DATE": "ds"})
        .sort_values("ds")
    )

    ts = ts[ts["y"] > 50]
    ts = ts[ts["ds"] < ts["ds"].max()]

    if len(ts) < 24:
        continue

    # Rolling trend on actuals
    ts["Rolling_3M"] = ts["y"].rolling(3, center=True).mean()

    # Prophet model
    m = Prophet(interval_width=0.95, yearly_seasonality=True)
    m.fit(ts[["ds", "y"]])

    future = m.make_future_dataframe(periods=12, freq="ME")
    forecast = m.predict(future)
    
    # Merge predictions with actuals
    merged = ts.merge(
        forecast[["ds", "yhat"]],
        on="ds",
        how="left"
    )

    # Error metrics (in-sample)
    y_true = merged["y"]
    y_pred = merged["yhat"]

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    results.append({
        "Neighborhood": neigh,
        "Months": len(ts),
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2),
        "MAPE (%)": round(mape, 2),
    })

    # Split forecast portion
    fc = forecast[forecast["ds"] > ts["ds"].max()]
    
    cutoff = ts["ds"].max()

    combined = pd.concat([
        ts[["ds", "y"]].rename(columns={"y": "value"}),
        fc[["ds", "yhat"]].rename(columns={"yhat": "value"}),
    ]).sort_values("ds")

    combined["Rolling_3M_Full"] = (
        combined["value"]
        .rolling(window=3, min_periods=1, center=False)
        .mean()
    )

    # keep last 2 actual points so the first forecast rolling value is valid
    combined_plot = combined[combined["ds"] >= (cutoff - pd.offsets.MonthEnd(2))]
    combined_plot = combined_plot[combined_plot["ds"] > cutoff]


    fig = go.Figure()

    # Actuals
    fig.add_trace(go.Scatter(
        x=ts["ds"],
        y=ts["y"],
        mode="lines+markers",
        name="Actuals",
        line=dict(width=2)
    ))

    # Forecast line
    fig.add_trace(go.Scatter(
        x=fc["ds"],
        y=fc["yhat"],
        mode="lines",
        name="Forecast",
        line=dict(width=3)
    ))

    # Confidence interval band
    fig.add_trace(go.Scatter(
        x=pd.concat([fc["ds"], fc["ds"][::-1]]),
        y=pd.concat([fc["yhat_upper"], fc["yhat_lower"][::-1]]),
        fill="toself",
        fillcolor="rgba(255,255,255,0.12)",
        line=dict(color="rgba(255,255,255,0)"),
        hoverinfo="skip",
        name="95% CI"
    ))
    
    fig.add_trace(go.Scatter(
        x=combined_plot["ds"],
        y=combined_plot["Rolling_3M_Full"],
        mode="lines",
        name="3M Rolling Trend (Actuals → Forecast)",
        line=dict(width=2, dash="dash")
    ))

    fig.update_layout(
        title=f"Monthly 311 Complaint Forecast — {neigh}",
        template="plotly_dark",
        height=700,
        hovermode="x unified",
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font_color="#FFF",
        hoverlabel=dict(
            bgcolor="rgba(20,20,20,0.95)",
            font_color="#EAEAEA",
            bordercolor="#555",
            font_size=13,
            font_family="Inter, Arial, sans-serif",
        )
    )

    fig.show()


20:56:53 - cmdstanpy - INFO - Chain [1] start processing
20:56:53 - cmdstanpy - INFO - Chain [1] done processing


20:56:54 - cmdstanpy - INFO - Chain [1] start processing
20:56:54 - cmdstanpy - INFO - Chain [1] done processing


20:56:55 - cmdstanpy - INFO - Chain [1] start processing
20:56:55 - cmdstanpy - INFO - Chain [1] done processing


In [11]:
# Final accuracy results table
forecast_eval = pd.DataFrame(results).sort_values("MAPE (%)")
forecast_eval

,Neighborhood,Months,MAE,RMSE,MAPE (%)
1,Alief,107,122.88,151.69,11.50
2,Central Southwest,107,131.57,170.11,12.44
0,Greater Heights,107,167.23,229.91,13.95


## Citywide Resolution Time Forecast (Prophet)

**Purpose**

Forecast monthly average 311 resolution time (days) at the citywide level using a univariate Prophet time-series model.

**Data Conditioning**

- **Outlier handling:** cap `RESOLUTION_TIME_DAYS` at the 95th percentile to limit the influence of extreme cases
- **Leakage prevention:** exclude the two most recent months of data (often incomplete)
- **Missing resolution times:**
    - Impute missing values using the **median resolution time by `CASE TYPE`**, when available
    - Drop records where resolution time remains missing after imputation

**Time Series Construction**

- Convert `CREATED DATE` to month-end timestamps (`ds`)
- Aggregate to **monthly mean** resolution time
- Reindex to a complete month-end date range
- Fill missing months via linear interpolation with backfill/forward-fill to ensure continuity

**Target Variable**

- `y`: monthly mean `RESOLUTION_TIME_DAYS`

**Modeling Approach**

- Single **citywide Prophet model**
- Yearly seasonality enabled
- No external regressors included

**Evaluation**

- Last 6 observed months reserved as a holdout set
- Compute **MAPE** over the most recent **3 years of overlapping actuals and predictions**
- Evaluation is diagnostic and not used for model tuning

**Forecasting**

- Generate a 12-month forward forecast from the final observed month
- Forecast begins at the last actual data point to ensure visual continuity

**Visualization**

- Plot full historical monthly actuals
- Overlay forecasted values starting at the final observed month
- Display 95% confidence intervals around forecasted resolution times

**Notes**

- Interpolation is used solely to preserve temporal continuity, not to introduce signal
- Imputation strategy assumes case-type medians are stable over time
- Results should be interpreted alongside recent historical volatility and reported MAPE

In [12]:
cap = df["RESOLUTION_TIME_DAYS"].quantile(0.95)
df["RESOLUTION_TIME_DAYS"] = df["RESOLUTION_TIME_DAYS"].clip(upper=cap)

In [13]:
# create ds early (month-end)
df_model = df.copy()
df_model["ds"] = df_model["CREATED DATE"].dt.to_period("M").dt.to_timestamp("M")

# drop the most recent TWO months present in ds (matches precompute intent)
latest = df_model["ds"].max()
cutoff = (latest.to_period("M") - 1).to_timestamp("M")
df_model = df_model[df_model["ds"] < cutoff].copy()

In [14]:
# Median per case type (can contain NaN)
medians_by_type = df_model.groupby('CASE TYPE')['RESOLUTION_TIME_DAYS'].median()

# Map case-type median to rows
df_model['MEDIAN_BY_TYPE'] = df_model['CASE TYPE'].map(medians_by_type)

# Fill missing durations ONLY where case-type median exists
df_model['RESOLUTION_TIME_DAYS'] = df_model.apply(
    lambda row: row['MEDIAN_BY_TYPE'] if pd.isna(row['RESOLUTION_TIME_DAYS']) and not pd.isna(row['MEDIAN_BY_TYPE'])
    else row['RESOLUTION_TIME_DAYS'],
    axis=1
)

# Drop rows where RESOLUTION_TIME_DAYS is still NaN AND the case type median was also NaN
df_model = df_model.dropna(subset=['RESOLUTION_TIME_DAYS'])

In [21]:
ts = (
    df_model
    .groupby("ds")["RESOLUTION_TIME_DAYS"]
    .mean()
    .reset_index()
    .rename(columns={"RESOLUTION_TIME_DAYS": "y"})
)

full_range = pd.date_range(ts["ds"].min(), ts["ds"].max(), freq="ME")
ts = ts.set_index("ds").reindex(full_range).rename_axis("ds").reset_index()
ts["y"] = ts["y"].interpolate().bfill().ffill()

test = ts.tail(6).copy()
train = ts.iloc[:-6].copy()
train = train[["ds", "y"]]

model = Prophet(yearly_seasonality=True)
model.fit(train)

future = model.make_future_dataframe(periods=12, freq="ME")
forecast = model.predict(future)

pred = forecast[["ds", "yhat"]].merge(test[["ds", "y"]], on="ds", how="inner")

merged = forecast[["ds", "yhat"]].merge(ts[["ds", "y"]], on="ds", how="left")

cutoff = merged["ds"].max() - pd.DateOffset(years=3)
recent = merged[merged["ds"] >= cutoff].dropna(subset=["y", "yhat"])
recent = recent[recent["y"] > 0]

mape = (abs(recent["y"] - recent["yhat"]) / recent["y"]).mean() * 100
print(f"MAPE (3y in-sample): {mape:.2f}%")

# Plot
last_actual = ts["ds"].max()

# Actuals (full history)
fig = px.line(
    ts,
    x="ds",
    y="y",
    title="Resolution Time Forecast (Actuals + Forecast)",
    labels={"y": "Resolution Days"},
    template="plotly_dark"
)

# Forecast line starting from last actual month (include that last month point)
forecast_from_last = forecast[forecast["ds"] >= last_actual]

fig.add_scatter(
    x=forecast_from_last["ds"],
    y=forecast_from_last["yhat"],
    mode="lines+markers",
    name="Forecast",
)

# CI band
fig.add_scatter(
    x=pd.concat([forecast_from_last["ds"], forecast_from_last["ds"][::-1]]),
    y=pd.concat([forecast_from_last["yhat_upper"], forecast_from_last["yhat_lower"][::-1]]),
    fill="toself",
    fillcolor="rgba(255,255,255,0.12)",
    line=dict(color="rgba(255,255,255,0)"),
    hoverinfo="skip",
    name="95% CI"
)

fig.update_layout(
    height=700,
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    font_color="#FFF",
    hovermode="x unified",
    hoverlabel=dict(
            bgcolor="rgba(20,20,20,0.95)",
            font_color="#EAEAEA",
            bordercolor="#555",
            font_size=13,
            font_family="Inter, Arial, sans-serif",
        )
)

fig.show()

21:29:50 - cmdstanpy - INFO - Chain [1] start processing
21:29:50 - cmdstanpy - INFO - Chain [1] done processing


MAPE (3y in-sample): 12.63%


## Neighborhood-Level Resolution Time Forecasts

**Purpose**

Forecast monthly average 311 resolution time for the highest-volume neighborhoods using independent time-series models.

**Data Selection**

- Identify the top 3 neighborhoods by total complaint volume
- Filter records to each neighborhood independently
- Exclude low-activity months using a **citywide minimum volume threshold** (`MIN_REPORTS = 1000`)
- Require at least **24 valid months** per neighborhood to model

**Target Variable**

- `y`: monthly mean `RESOLUTION_TIME_DAYS`

**Time Series Construction**

- Convert `CREATED DATE` to month-end timestamps
- Aggregate resolution time to monthly averages
- Restrict months to those meeting the global volume threshold
- Reindex to a complete monthly range within the valid window
- Fill missing months via linear interpolation with backfill/forward-fill

**Modeling Approach**

- One **independent Prophet model per neighborhood**
- Yearly seasonality enabled
- No pooled learning across neighborhoods

**Train / Test Split**

- Final 6 observed months held out for evaluation
- Remaining history used for training

**Evaluation**

- Compute **MAPE** on the 6-month holdout window
- Report neighborhood-specific forecast accuracy

**Forecasting**

- Generate a 12-month forward forecast from the final observed month
- No external regressors are included in the current implementation

**Visualization**

- Plot full historical monthly resolution time
- Overlay forecast beginning at the last observed month
- Display 95% confidence intervals around forecasted values
- Annotate plots with neighborhood-level MAPE

**Notes**

- Forecast reliability varies by neighborhood history length and volatility
- Interpolation is used strictly for continuity, not signal creation
- Results should be interpreted alongside reported MAPE values

In [ ]:
MIN_REPORTS = 1000

df_model = df_model.copy()
df_model["ds"] = df_model["CREATED DATE"].dt.to_period("M").dt.to_timestamp("M")

month_vol = (
    df_model.groupby("ds")
            .size()
            .reset_index(name="volume")
            .sort_values("ds")
)
valid_months = set(month_vol.loc[month_vol["volume"] >= MIN_REPORTS, "ds"])

for neigh in top_3_neigh:
    df_nbh = df_model[df_model["NEIGHBORHOOD"] == neigh].copy()
    if df_nbh.empty:
        continue

    ts = (
        df_nbh.groupby("ds")["RESOLUTION_TIME_DAYS"]
              .mean()
              .reset_index()
              .rename(columns={"RESOLUTION_TIME_DAYS": "y"})
              .sort_values("ds")
    )

    # Drop low-volume months (citywide rule)
    ts = ts[ts["ds"].isin(valid_months)].copy()

    if len(ts) < 24:
        continue

    full_range = sorted([d for d in valid_months if ts["ds"].min() <= d <= ts["ds"].max()])
    ts = (
        ts.set_index("ds")
          .reindex(full_range)
          .rename_axis("ds")
          .reset_index()
    )
    ts["y"] = ts["y"].interpolate().bfill().ffill()

    # holdout split (example)
    test = ts.tail(6).copy()
    train = ts.iloc[:-6].copy()

    model = Prophet(yearly_seasonality=True)
    model.fit(train[["ds", "y"]])

    future = model.make_future_dataframe(periods=12, freq="ME")
    forecast = model.predict(future)

    pred = forecast[["ds", "yhat"]].merge(test[["ds", "y"]], on="ds", how="inner")
    if not pred.empty:
        mape = mean_absolute_percentage_error(pred["y"], pred["yhat"]) * 100
        print(f"{neigh} — MAPE: {mape:.2f}%")

    fig = px.line(
        ts, x="ds", y="y",
        title=f"Resolution Time Forecast — {neigh} (MAPE {mape:.1f}%)",
        labels={"y": "Resolution Days"},
        template="plotly_dark"
    )

    fig.add_scatter(
        x=forecast_from_last["ds"],
        y=forecast_from_last["yhat"],
        mode="lines+markers",
        name="Forecast",
    )

    fig.add_scatter(
        x=pd.concat([forecast_from_last["ds"], forecast_from_last["ds"][::-1]]),
        y=pd.concat([forecast_from_last["yhat_upper"], forecast_from_last["yhat_lower"][::-1]]),
        fill="toself",
        fillcolor="rgba(255,255,255,0.12)",
        line=dict(color="rgba(255,255,255,0)"),
        hoverinfo="skip",
        name="95% CI"
    )

    fig.update_layout(
        height=700,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font_color="#FFF",
        hovermode="x unified",
        hoverlabel=dict(
            bgcolor="rgba(20,20,20,0.95)",
            font_color="#EAEAEA",
            bordercolor="#555",
            font_size=13,
            font_family="Inter, Arial, sans-serif",
        )
    )

    fig.show()


21:27:29 - cmdstanpy - INFO - Chain [1] start processing
21:27:29 - cmdstanpy - INFO - Chain [1] done processing


Greater Heights — MAPE: 23.70%


21:27:30 - cmdstanpy - INFO - Chain [1] start processing
21:27:30 - cmdstanpy - INFO - Chain [1] done processing


Alief — MAPE: 19.11%


21:27:30 - cmdstanpy - INFO - Chain [1] start processing
21:27:30 - cmdstanpy - INFO - Chain [1] done processing


Central Southwest — MAPE: 23.66%
